# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db
#%config SqlMagic.style = 'table'

Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

In [2]:
%%sql
SELECT name
FROM sqlite_master

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
sqlite_autoindex_get_fit_now_member_1
get_fit_now_check_in
solution
check_solution
income



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [5]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


In [16]:
%config SqlMagic.displaylimit = 10
#see full text

In [28]:
%%sql
SELECT *
FROM crime_scene_report
WHERE type="murder" AND date="20180115" AND city="SQL City"

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [48]:
%%sql
SELECT *
FROM person
WHERE address_street_name = "Franklin Ave" AND name LIKE "%Annabel%";

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [42]:
%%sql
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr" 
ORDER BY address_number DESC 
LIMIT 1

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [47]:
%%sql
SELECT *
FROM interview
WHERE person_id IN ("16371","14887")

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [50]:
%%sql 
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE "%48Z%" AND check_in_date = "20180109"

Running query in 'sqlite:///sql-murder-mystery.db'

membership_id,check_in_date,check_in_time,check_out_time
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700


In [53]:
%%sql 
SELECT *
FROM get_fit_now_member
WHERE id IN ("48Z7A","48Z55")

Running query in 'sqlite:///sql-murder-mystery.db'

id,person_id,name,membership_start_date,membership_status
48Z55,67318,Jeremy Bowers,20160101,gold
48Z7A,28819,Joe Germuska,20160305,gold


In [68]:
%%sql 
SELECT *
FROM person
JOIN drivers_license ON person.license_id = drivers_license.id
WHERE name IN ("Jeremy Bowers", "Joe Germuska")
--omg Joe doesnt have a driver's license' and his number plate contains "H42W"

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn,id_1,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


Jeremy Bowers is the killer !!


### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [95]:
%%sql
INSERT INTO solution VALUES (1, 'Miranda Priestly');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"


In [73]:
%%sql
SELECT *
FROM interview
WHERE person_id ="67318"

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [94]:
%%sql
SELECT DISTINCT *
FROM person 
JOIN drivers_license ON person.license_id = drivers_license.id
INNER JOIN income ON person.ssn = income.ssn --JOIN same as INNER JOIN
JOIN facebook_event_checkin ON person.id = facebook_event_checkin.person_id 
WHERE hair_color ="red" AND car_make="Tesla" AND gender="female"


Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn,id_1,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,ssn_1,annual_income,person_id,event_id,event_name,date
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171206
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171212
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171229


OMG it's Miranda Priestly !! She is the mastermind behind the murder.